In [ ]:
!pip install sklearn
!pip install tensorflow
!pip install keras
!pip install tqdm
!pip install matplotlib


In [6]:
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np 
from glob import glob 

In [7]:
def load_dataset(data_path, shuffle=None):
    kwargs = {}
    if shuffle != None:
        kwargs['shuffle'] = shuffle
    data = load_files(data_path, **kwargs)
    img_files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 3)
    return img_files, targets

train_files, train_targets = load_dataset('data/train')
valid_files, valid_targets = load_dataset('data/valid')
test_files, test_targets = load_dataset('data/test', shuffle=False)

# load lables
label_name = [item[11:-1] for item in sorted(glob("data/train/*/"))]

print('train_files size: {}'.format(len(train_files)))
print('train_files shape: {}'.format(train_files.shape))
print('target shape: {}'.format(train_targets.shape))
print(label_name)

train_files size: 2000
train_files shape: (2000,)
target shape: (2000, 3)
['melanoma', 'nevus', 'seborrheic_keratosis']


In [8]:
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from tqdm import tqdm 
def path_to_tensor(img_path):
    img = image.load_img(img_path,target_size=(384,256))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)
def paths_to_tensor(image_paths):
    return np.vstack([path_to_tensor(path)for path in image_paths])


In [9]:
#loading images into tensors 
train_tensors = paths_to_tensor(tqdm(train_files))

valid_tensors = paths_to_tensor(tqdm(valid_files))

test_tensors = paths_to_tensor(tqdm(test_files))


print(train_tensors.shape)

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [03:24<00:00,  2.93it/s]


(2000, 384, 256, 3)


In [10]:
import matplotlib.pyplot as plt 
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
apply_train_image_tranform = False 
if apply_train_image_tranform:
    datagen_train = ImageDataGenerator(horizontal_flip=True
                                      ,vertical_flip=True)
    datagen_train.fit(train_tensors)
    shape = (train_tensors.shape[0]*2,)+train_tensors.shape[1:]
    generated = np.ndarray(shape=shape)
    for i , image in tqdm(enumerate(train_tensors)):
        generated[i]=datagen_train.random_transform(image)
        train_tensors=np.concatenate((train_tensors,generated))
        train_targets=train_targets.repeat(2, axis=0)
train_imgs_preprocess = preprocess_input(train_tensors)
valid_imgs_preprocess = preprocess_input(valid_tensors)
test_imgs_preprocess = preprocess_input(test_tensors)
del train_tensors, valid_tensors, test_tensors

    

In [11]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
transfer_model = InceptionResNetV2(include_top=False)
train_data = transfer_model.predict(train_imgs_preprocess)
valid_data = transfer_model.predict(valid_imgs_preprocess)
test_data = transfer_model.predict(test_imgs_preprocess)
del train_imgs_preprocess,valid_imgs_preprocess,test_imgs_preprocess
print(train_data.shape)


(2000, 10, 6, 1536)


In [12]:
import gc
gc.collect()


2157

In [23]:
from keras.layers import Conv2D, Dropout, Flatten, Dense, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential

BCD = Sequential()
BCD.add(GlobalAveragePooling2D(input_shape=train_data.shape[1:]))
BCD.add(Dropout(0.2))
BCD.add(Dense(2048,activation='relu'))
BCD.add(Dropout(0.3))
BCD.add(Dense(3,activation='softmax'))
BCD.summary()


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 1536)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1536)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 2048)              3147776   
_________________________________________________________________
dropout_3 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 6147      
Total params: 3,153,923
Trainable params: 3,153,923
Non-trainable params: 0
_________________________________________________________________


In [24]:

BCD.compile(loss='categorical_crossentropy',optimizer='adam',
           metrics=['accuracy'])


In [25]:
from keras.callbacks import ModelCheckpoint
import os 
checkpoint_filepath = 'weights.hdf5'
checkpointer = ModelCheckpoint(
    filepath=checkpoint_filepath,
    verbose=1,save_best_only=True)
BCD.fit(train_data,train_targets,
        validation_data=(valid_data,valid_targets),
       epochs=60,batch_size=1,
        callbacks=[checkpointer],verbose=1)

Epoch 1/60
1998/2000 [============================>.] - ETA: 0s - loss: 1.1430 - accuracy: 0.6396 ETA: 0s
Epoch 00001: val_loss improved from inf to 1.40694, saving model to weights.hdf5
2000/2000 [==============================] - 36s 18ms/step - loss: 1.1426 - accuracy: 0.6395 - val_loss: 1.4069 - val_accuracy: 0.5200
Epoch 2/60
2000/2000 [==============================] - ETA: 0s - loss: 0.7690 - accuracy: 0.6895
Epoch 00002: val_loss improved from 1.40694 to 1.29560, saving model to weights.hdf5
2000/2000 [==============================] - 36s 18ms/step - loss: 0.7690 - accuracy: 0.6895 - val_loss: 1.2956 - val_accuracy: 0.5333
Epoch 3/60
1998/2000 [============================>.] - ETA: 0s - loss: 0.7336 - accuracy: 0.6817
Epoch 00003: val_loss improved from 1.29560 to 0.80606, saving model to weights.hdf5
2000/2000 [==============================] - 37s 18ms/step - loss: 0.7333 - accuracy: 0.6820 - val_loss: 0.8061 - val_accuracy: 0.6667
Epoch 4/60
1999/2000 [====================

Epoch 29/60
1998/2000 [============================>.] - ETA: 0s - loss: 0.5460 - accuracy: 0.7548
Epoch 00029: val_loss did not improve from 0.64359
2000/2000 [==============================] - 37s 18ms/step - loss: 0.5455 - accuracy: 0.7550 - val_loss: 0.7129 - val_accuracy: 0.7467
Epoch 30/60
1999/2000 [============================>.] - ETA: 0s - loss: 0.5123 - accuracy: 0.7919
Epoch 00030: val_loss did not improve from 0.64359
2000/2000 [==============================] - 37s 19ms/step - loss: 0.5127 - accuracy: 0.7915 - val_loss: 0.6983 - val_accuracy: 0.7400
Epoch 31/60
1998/2000 [============================>.] - ETA: 0s - loss: 0.5194 - accuracy: 0.7763
Epoch 00031: val_loss did not improve from 0.64359
2000/2000 [==============================] - 38s 19ms/step - loss: 0.5202 - accuracy: 0.7760 - val_loss: 0.8899 - val_accuracy: 0.6933
Epoch 32/60
1998/2000 [============================>.] - ETA: 0s - loss: 0.5460 - accuracy: 0.7703
Epoch 00032: val_loss did not improve from 0.6

Epoch 58/60
2000/2000 [==============================] - ETA: 0s - loss: 0.4567 - accuracy: 0.8045
Epoch 00058: val_loss did not improve from 0.64359
2000/2000 [==============================] - 37s 18ms/step - loss: 0.4567 - accuracy: 0.8045 - val_loss: 0.9306 - val_accuracy: 0.7133
Epoch 59/60
1998/2000 [============================>.] - ETA: 0s - loss: 0.4482 - accuracy: 0.8018
Epoch 00059: val_loss did not improve from 0.64359
2000/2000 [==============================] - 37s 19ms/step - loss: 0.4479 - accuracy: 0.8020 - val_loss: 0.8911 - val_accuracy: 0.7400
Epoch 60/60
2000/2000 [==============================] - ETA: 0s - loss: 0.4612 - accuracy: 0.7990
Epoch 00060: val_loss did not improve from 0.64359
2000/2000 [==============================] - 37s 19ms/step - loss: 0.4612 - accuracy: 0.7990 - val_loss: 0.9591 - val_accuracy: 0.7000


In [27]:
BCD.load_weights(checkpoint_filepath)

In [28]:
import csv 
my_predictions = [BCD.predict(np.expand_dims(feature, axis=0)) for feature in test_data]



with open('my_transfer.csv', 'w', newline='') as csvfile:
    result_writger = csv.writer(csvfile)
    result_writger.writerow(['Id', 'task_1', 'task_2'])
    for test_filepath, test_prediction in zip(test_files, my_predictions):
        result_writger.writerow([test_filepath, test_prediction[0][0], test_prediction[0][2]])